In [1]:
from openai import OpenAI
import config
import json
import requests

In [2]:
client = OpenAI(
api_key=config.OPENAI_API_KEY,
)

In [4]:
products="""
1. Producto: 4k television LG
    Categoría: Televisión
    Manufacturer: LG
    Price: 500

2. Producto: 4k television Samsung
    Categoría: Televisión
    Manufacturer: Samsung
    Price: 1000

3. Producto: Xiaomi Mi 13
    Categoría: Celular
    Manufacturer: Xiaomi
    Price: 900

4. Producto: Samsung Galaxy X
    Categoría: Celular
    Manufacturer: Samsung
    Price: 1000

5. Producto: Xiaomi Tab
    Categoría: Tablet
    Manufacturer: Samsung
    Price: 300

6. Producto: Samsung Galaxy Tab
    Categoría: Tablet
    Manufacturer: Samsung
    Price: 500
"""

tab="""
1. acción: Saludar a alguien
    Comando: [Hola mundo]

2. acción: Correr o trotar
    Comando: [running_boy]
"""


delimiter = "####"
system_message = f"""
Eres un bot muy cordial, dispuesto a responder todas las solicitudes que recibas.

Siga estos pasos para responder a las consultas de los usuarios de la manera más amable posible.
La consulta del usuario se delimitará con cuatro símbolos de hashtag, es decir, {delimiter}. La respuesta debe tener una longitud máxima de 20 palabras.

Paso 1:{delimiter} Primero debes identificar si el usuario está preguntando sobre un producto o productos.

Paso 2:{delimiter} Si el usuario pregunta por productos específicos, identifique si los productos están \
en la siguiente lista.
Todos los productos disponibles: 
{products}
Si el usuario quiere accionar un comando específico, búscalo en la siguiente lista:{tab}
Si el usuario pregunta por productos en general, solo menciona los nombres de los productos y evita datos adicionales.

Paso 3:{delimiter} Si el mensaje contiene productos \
de la lista anterior, enumere las suposiciones que el usuario \
en su mensaje \
por ejemplo, que la garantía del producto X es más larga que el producto Y, o que el producto Z tiene un rating mayor a \
4 puntos.

Si el mensaje contiene un comando, identificarlo junto con su descripción.

Paso 4:{delimiter}:Si hay un comando de pestaña identificado concatena en tu mensaje habitual, el comando correspondiente.

Utilice el siguiente formato:
Paso 1:{delimiter} <paso 1 razonamiento>
Paso 2:{delimiter} <paso 2 razonamiento>
Paso 3:{delimiter} <paso 3 razonamiento>
Paso 4:{delimiter} <paso 4 razonamiento>
La respuesta debe estar separada de los comandos por el delimitador "$$".
Respuesta al usuario:{delimiter} <respuesta al cliente {"$$"} comandos>
En caso no se encuentren comandos para la consulta, la respuesta debe tener el formato de:
Respuesta al usuario:{delimiter} <respuesta al cliente {"$$"} Null>


Asegúrese de incluir el delimitador {delimiter} para separar cada paso y que la respuesta al usuario sea de maximo de 20 palabras.

Recuerda que siempre debes dar tus respuesta en lenguaje español
"""

In [5]:
def query_chatgpt(messages):
    '''Send messages to OpenAI's Chat GPT model and get a response.'''
    #You can change the model name for better performance (gpt-3.5-turbo, gpt-4 or gpt-4-1106-preview)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125", 
        messages=messages,
    )
    response_message = response.choices[0].message.content
    return response_message

In [6]:
messages = [
    {"role": "system", "content":system_message}
]

In [7]:
def conversation(prompt):
    global messages
    messages.append({"role": "user", "content": prompt})
    response_message = query_chatgpt(messages)
    
    #play_audio(text_to_speech(response_message, audio_file), text_speed)
    messages.append({"role": "assistant", "content": response_message})
    output = ''
    #print(messages)
    for message in messages:
        if message['role'] != 'system':
            #output += message['role'] + ': ' + message['content'] + '\n\n'
            try:
                final_reponse= message['content'].split(delimiter)[-1].strip()
                output += message['role'] + ': ' + final_reponse + '\n\n'
            except Exception as e:
                print(e)
                output = "Perdón, no entiendo"
    return final_reponse

In [7]:
conversation("hola")

'¡Hola! ¿En qué puedo ayudarte hoy? $$ Null'

In [8]:
conversation("¿Que me puedes contar?")

'¿Te gustaría saber más sobre nuestros productos de tecnología? $$ Null'

In [9]:
conversation("Dime tus productos")

'Tenemos televisores, celulares y tablets en nuestra tienda. $$ Null'

In [10]:
conversation("Enumera los productos disponibles")

'4k television LG, 4k television Samsung, Xiaomi Mi 13, Samsung Galaxy X, Xiaomi Tab, Samsung Galaxy Tab. $$ Null'

In [11]:
conversation("Háblame acerca de las televisiones")

'Tenemos 4k television LG con un precio de 500 y 4k television Samsung por 1000. $$ Null'

In [12]:
conversation("Dame más detalles")

'¿Te gustaría más detalles sobre la 4k television LG o la 4k television Samsung? $$ Null'

In [13]:
conversation("4k television LG")

'La 4k television LG tiene un precio de 500. $$ Null'

In [14]:
conversation("¿Y celulares?")

'Tenemos Xiaomi Mi 13 y Samsung Galaxy X en celulares. $$ Null'

In [8]:
conversation("Ahora dame el comando para saludar")

'El comando para saludar es [Hola mundo] $$ [Hola mundo]'

In [9]:
conversation("Ahora quiero correr")

'El comando para correr es [running_boy] $$ [running_boy]'